<h1> BPR algorithm for individual recommendations </h1>
<h6> All .py files in the bpr_files folder are from https://github.com/shah314/BPR and its parts have been adapted for the given application and dataset in the following .ipynb file </h6>

In [36]:
"""
Created on Sat Sep 12 12:46:51 2020

@author: shah
"""

from filereader import read_ratings, read_movies
from bpr import bpr_update
from hitrate import hit_rate
from util import cov_matrix
from util import random_vector
import numpy as np
import pandas as pd
from classes import user
from classes import movie
from util import min_rating, random_vector, num_users
from random import seed


<h2> Read user ratings from dataset with #apps/user >= 20 and #appear/job >=20, to ensure dense dataset </h2>

In [37]:
#Adapted from filereader file for a given application/dataset

r_cols = ['UserID','JobID','Rating']
filename = r"C:\Users\weron\OneDrive\Dokumenty\RecSys_group5\dataset\working\user_ratings_neg_1000_20_20_1_train.csv"
ratingsTr = pd.read_csv(filename,  sep=',', names=r_cols, encoding='latin-1',skiprows=[0]) #Train set
filename1 = r"C:\Users\weron\OneDrive\Dokumenty\RecSys_group5\dataset\working\user_ratings_neg_1000_20_20_1_test.csv"
ratingsT = pd.read_csv(filename1,  sep=',', names=r_cols, encoding='latin-1',skiprows=[0]) #Test set
print(ratingsT.head())
ratingsTr['UserID'] = ratingsTr['UserID'].astype('int')
ratingsTr['JobID'] = ratingsTr['JobID'].astype('int')
ratingsTr['Rating'] = ratingsTr['Rating'].astype('float')

ratingsT['UserID'] = ratingsT['UserID'].astype('int')
ratingsT['JobID'] = ratingsT['JobID'].astype('int')
ratingsT['Rating'] = ratingsT['Rating'].astype('float')

print(ratingsTr.head())

users = dict()
testcount = 0
traincount = 0
trainuserdict = dict()

for index, row in ratingsTr.iterrows():
    userid = int(row['UserID'])
    movieid = int(row['JobID'])
    rating1 = float(row['Rating'])
    traincount = traincount + 1
    if userid in users.keys():
        user1 = users[userid]
        user1.movies_train[movieid] = rating1
    else:
        user1 = user(userid)
        user1.factor = random_vector()
        user1.movies_train[movieid] = rating1
        users[userid] = user1
        trainuserdict[userid] = 1

for index, row in ratingsT.iterrows():
    userid = int(row['UserID'])
    movieid = int(row['JobID'])
    rating1 = float(row['Rating'])     
    testcount = testcount + 1
    if userid in users.keys():
        user1 = users[userid]
        user1.movies_test[movieid] = rating1
    else:
        user1 = user(userid)
        user1.factor = random_vector()
        user1.movies_test[movieid] = rating1
        users[userid] = user1

for index, row in ratingsT.iterrows():
    userid = int(row['UserID'])
    movieid = int(row['JobID'])
    rating1 = float(row['Rating'])
    if userid in users.keys():
        user1 = users[userid]
        user1.movies_all[movieid] = rating1
for index, row in ratingsTr.iterrows():
    userid = int(row['UserID'])
    movieid = int(row['JobID'])
    rating1 = float(row['Rating'])
    if userid in users.keys():
        user1 = users[userid]
        user1.movies_all[movieid] = rating1

print(testcount)

   UserID    JobID  Rating
0     554   283949       1
1     554   251966       1
2     769   551465       1
3     769  1114487      -1
4     769   478877       1
   UserID   JobID  Rating
0     554  640492     1.0
1     554  280275     1.0
2     554  747584     1.0
3     554   25820     1.0
4     554  600058     1.0
30991


In [38]:
print(traincount)
#print(users)
colTr = ratingsTr['JobID']
colT = ratingsT['JobID']
print(colTr)
print(colT)

241849
0          640492
1          280275
2          747584
3           25820
4          600058
           ...   
241844    1103712
241845     338735
241846    1042356
241847    1114014
241848    1113501
Name: JobID, Length: 241849, dtype: int32
0         283949
1         251966
2         551465
3        1114487
4         478877
          ...   
30986    1112114
30987    1115960
30988     813176
30989     674546
30990     103118
Name: JobID, Length: 30991, dtype: int32


In [39]:
seed(42)
np.random.seed(42)
print("Reading jobs...")
filename = r"C:\Users\weron\OneDrive\Dokumenty\RecSys_group5\dataset\jobs.tsv"
r_cols = ['JobID',	'WindowID',	'Title',	'Description',	'Requirements'	'City'	,'State',	'Country',	'Zip5'	,'StartDate'	,'EndDate']
df = pd.read_table(filename, sep="\t", encoding='latin-1',  on_bad_lines='skip', low_memory=False)
print(df['JobID'])
print(df.head())


Reading jobs...
0                1
1                4
2                7
3                8
4                9
            ...   
1091918    1116242
1091919    1116253
1091920    1116294
1091921    1116304
1091922    1116307
Name: JobID, Length: 1091923, dtype: int64
   JobID  WindowID                             Title  \
0      1         1  Security Engineer/Technical Lead   
1      4         1         SAP Business Analyst / WM   
2      7         1     P/T HUMAN RESOURCES ASSISTANT   
3      8         1            Route Delivery Drivers   
4      9         1                      Housekeeping   

                                         Description  \
0  <p>Security Clearance Required:&nbsp; Top Secr...   
1  <strong>NO Corp. to Corp resumes&nbsp;are bein...   
2  <b>    <b> P/T HUMAN RESOURCES ASSISTANT</b> <...   
3  CITY BEVERAGES Come to work for the best in th...   
4  I make  sure every part of their day is magica...   

                                        Requirements      

In [41]:
print(df.iloc[1]['JobID'])
print(ratingsT['JobID'])
print(len(ratingsT), len(ratingsTr))

4
0         283949
1         251966
2         551465
3        1114487
4         478877
          ...   
30986    1112114
30987    1115960
30988     813176
30989     674546
30990     103118
Name: JobID, Length: 30991, dtype: int32
30991 241849


In [42]:
movies = dict()
for index, row in df.iterrows():
    movieid = row['JobID']
    movie1 = movie(movieid, 0)
    movie1.factor = random_vector()
    movies[movieid] = movie1
#movies = read_movies(r"C:\Users\weron\OneDrive\Dokumenty\RecSys_group5\dataset\jobs.tsv")
print("Finished Reading files.")

Finished Reading files.


In [44]:
from classes import user
print(len(movies))
u = users[554]
print(u.movies_train.keys())

#rand_neg = random.sample(movies.keys(), 1)[0]
print(len(users))

1091923
dict_keys([640492, 280275, 747584, 25820, 600058, 196603, 1113088, 40, 627377, 957, 600045, 283948, 812337, 300053, 273293, 817048, 802921, 491965])
10943


In [50]:
from bpr import bpr_update
from hitrate import hit_rate
iterations = 10

for i in range(iterations):
    
    print("Iteration " + str(i))
    bpr_update(users, movies)

print("Finished updating.")
print("Calculating Hit Rate...")


hits, denom, rms, rmsall = hit_rate(users, movies)

print("Trained:")
print("HitRate" + str(hits/denom))
print("Number of Users:" + str(len(users)))
print("Number of Movies:" + str(len(movies)))


Iteration 0
[79.29799697 79.96873925 79.23836888 78.40401147 79.31388204 79.68949796
 80.06560047 78.74428281 79.6388777  79.85333067 79.6418468  79.56598552
 79.29879354 78.92407747 79.07574862 79.33669147 78.96838413 79.85053485
 80.06172398 78.92680358 79.95737248 79.12442026 79.75135171 80.13922699
 79.21300793 78.6953595  78.88911786 79.14404771 79.50485711 79.76841172
 79.5244231  79.77671449 79.08587667 79.26699074 79.72421756 79.05810994
 79.4971499  79.38097291 79.1599868  79.37760846 79.51497245 79.13972316
 79.00268279 80.05698604 79.86462344 79.45075325 78.88002882 79.3116826
 79.12325061 79.8905728 ]
[75.15628347 75.31311358 74.9369932  74.13289985 74.22306174 74.54433195
 73.8440929  75.38405215 74.93284414 75.59687898 75.69390196 75.24526993
 75.29241583 74.63310219 75.63556931 74.6509375  75.03885497 75.58986732
 74.70041752 74.64702434 74.76091163 75.68636016 74.89768052 75.1574379
 74.62135184 75.6112809  74.55391689 74.5467104  74.16063623 75.04451859
 75.30557122 75

Traceback (most recent call last):
  File "c:\Users\weron\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\weron\AppData\Local\Temp\ipykernel_36976\4089645782.py", line 8, in <module>
    bpr_update(users, movies)
  File "c:\Users\weron\OneDrive\Dokumenty\RecSys_group5\bpr_files\bpr.py", line 37, in bpr_update
  File "c:\Users\weron\OneDrive\Dokumenty\RecSys_group5\bpr_files\bpr.py", line 61, in calculate_first_term
    negxuij = (boughtdot - notboughtdot) * -1
  File "<__array_function__ internals>", line 180, in dot
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\weron\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\weron